# FT8 Coding

The FT8 digital mode uses a variety of codes to enable reliable transmission of information in the presence of noise.
These include:

1. A LDPC (low-density parity-check) code which is used for error detection and correction.
1. A CRC (cyclic redundancy check) code which is used for error detection.
1. A Gray code which is used to minimise the introduction of errors when converting from the analog to the digital domain.

This notebook captures some background on each of these codes.

## LDPC Codes

In order to understand LDPC codes we first need a little bit of background on block codes.

### What is a Block Coding?

Block coding is a technique used to detect and sometimes correct errors that may occur when sending information 
in the form of digital bits across a communication channel.
Practical block codes that could both detect and correct errors were first developed in the late 1940s
by Golay [[1](#References)] and Hamming [[2](#References)].

Block coding operates using the following process:

1. The information to be transmitted is broken into fixed size blocks each containing $k$ bits of information.
1. An encoder takes the $k$ bits of information in each block and generates $r$ redundant bits which are then combined with the original $k$ bits of information to create an $n$ bit codeword where $n = k+r$.
1. The $n$ bit codeword is transmitted across the noisy communication channel using an appropriate modulation and demodulation method.
1. A decoder takes the output of the demodulator and attempts to reconstruct the original $k$ bits of information using the the received $n$ bit codeword which may have been corrupted by the noisy communication channel.

The performance of a particular block code can be described using several different measures:

1. The bandwidth efficiency or rate of the code, which a measure of how much the communication channel bandwidth needs to increase in order to accomodate the $r$ bits of redundant information added by the encoder - this is expressed by the the ratio $k/n$ where $k$ is the number of information bits in each block and $n$ is the number
of bits in the coresponding codeword for each block
1. The complexity of the encoding and decoding processes, which is a measure of how much computation is required to perform encoding and decoding operations
1. The error rate at the output of the decoder as a function of the communication channel signal to noise ratio - this is usually expressed by a graph of decoder output bit error rate vs communication channel signal to noise ratio for a specific type of communication channel

The goal of block code design is to optimise the code rate (make it high as possible), complexity (make it low as possible) and error rate (make it low as possible) within the constraints of a given system.

### What is a LDPC Code?

LDPC (low-density parity-check) codes are a family of parity-checking linear block codes first described
in the early 1960s by Gallager [[3](#References)].
Gallager original thesis included details on how both LDPC encoders and LDPC decoders could be constructed.
However, in the early 1960s, cost effective construction of systems based on LDPC codes (and the simulations needed to design them) was beyond the capability of the available technology.
As a consequence, LDPC codes were mostly forgotten for many years before being rediscovered in the 1990s
[[4](#References)].
LDPC are now widely used, having been incorporated into several communications standards.

Parity-checking linear block codes are defined by a series of linear parity-check equations that relate the $r$ redundant parity bits to the $k$ information bits in each $n$ bit codeword.
Each linear equation involves the addition (modulo 2) of a subset of the $k$ information bits and a subset of
the $r$ redundant parity bits such that the addition (modulo 2) equals zero.

These linear parity-check equations can by expressed in matrix form using a parity-check matrix that is designated in the literature by the symbol $H$.
The parity-check matrix has $m$ rows and $n$ columns, where $m$ is the number of parity-check equations and
$n$ is the number of bits in each codeword.
The value of $m$ must be >= $r$ in order to uniquely determine the $r$ parity bits.
Often $m = r$, as is the case for the LDPC code used by the FT8 data mode.
For each equation which corresponds to a matrix row, there is a one in the matrix column where a bit in the codeword is included in the equation and a zero in the matrix column where the bit is not included in the equation.

For LDPC codes, the parity-check matrix $H$ is sparse (i.e. contains many zeros) and therefore has a low-density of ones, which is where the "low-density" part of the LDPC name comes from.

There are many possible LDPC codes, that can each be defined by block size $k$, codeword size $n$
and parity-check matrix $H$.
System designers choose values for $k$, $n$ and $H$ in order to meet the performance objectives
of the system they are designing.

The designers of the FT8 digital mode have choosen a specific LDPC code with $k=91$ and $n=174$
and a parity-check matrix that is documented in the WSJT-X source code in the `lib/ft8/ldpc_174_91_c_parity.f90` compilation unit.
There are $83$ parity-check equations and each equation only includes $6$ or $7$ bits from the $174$ bit
codeword.
The parity-check matrix $H$ is therefore quite sparse.

The $91$ information bits in each block correspond to a single FT8 message that is first packed into $77$ bits,
to which a $14$ bit cyclic redundancy check outer code is then added.
The LDPC encoder then adds a further $83$ redundant parity bits to create a $174$ bit codeword for transmission.

The rate of the resulting LDPC code is $91/174$ or slightly more than $1/2$.
The rate of the combined LDPC code and CRC outer code is $77/174$ or slightly less than $1/2$.

### LDPC Encoder

The role of the encoder is to determine the $r$ parity bits from the $k$ information bits so that the a $n$ bit codeword can be constructed and passed onto the modulator for transmission across the communications channel.

For LDPC codes, this can be achieved by solving the parity check equations for the $r$ parity bits.
This can be done by starting with the parity check matrix $H$ and using linear algebra methods to obtain a generator matrix, which is typically denoted by the symbol $G$ in the literature.
The generator matrix $G$ has $r$ rows and $k$ columns and represents $r$ equations, one for each of the parity bits that must be generated.

Once the generator matrix $G$ has been determined for a particular LDPC code, the encoding step can be performed for each block by multiplying the generator matrix $G$ by a column vector containing the $k$ information bits in that block.

The FT8 protocol implementation in the WSJT-X program performs LDPC encoding in the `lib/ft8/encode174_91.f90` compilation unit, using a pre-calculated generator matrix $G$ that has $83$ rows and $91$ columns.
The pre-calculated generator matrix can be found in the `lib/ft8/ldpc_174_91_c_generator.f90` compilation unit.
The generator matrix $G$ is expressed as an array of $83$ hexadecimal strings, one for each row in the generator matrix $G$.
Each hexadecimal string has $23$ hex digits which is sufficient to represent $92$ bits.
As there are only $91$ columns in the generator matrix $G$, the last bit in each hexadecimal string is not used and is therefore always a zero.

### LDPC Decoder

The LDPC decoder is responsible for taking the output of the demodulator and determine the original $k$
information bits with as few errors as possible.

There are multiple methods that have been discovered for constructing LDPC decoders.
The decoding methods fall into two broad categories that depend on the type of input available to the decoder.

1. Hard decision decoders
1. Soft decision decoders

Hard decision decoders start with $n$ bit code words i.e. a hard decision has already been made by the detector
at the output of the demodulation process as to whether each bit in the codeword is a 1 or a 0 before the decoder starts its work to determine the $k$ information bits.

Soft decision decoders instead start with statistical probabilities that each of the $n$ codeword bits are a $0$ or $1$ and can typically provide the same error rate at lower signal to noise ratios than hard decision decoders.
However, this improvement comes at the cost of increased complexity.

Where sufficient computational capability exists to handle the complexity, soft decision decoders are generally preferred as they allow successful communication at lower signal to noise ratios i.e. the reduce the minimum signal to noise ratio that is required for near error free communication, typically by a few dB.

### Log Likelyhood Ratio

A statistic used as input to many soft decision decoders for LDPC codes is the log likelyhood ratio or LLR.

The LLR (as used in WSJT-X) is defined by the equation:

$${LLR}_n = \ln{(p_n(v|1)/p_n(v|0))} = \ln{(p_n(v|1))} - \ln{(p_n(v|0))}$$

where $p_n(v|0)$ is the probability density function of the received symbol observations $v$, 
conditional on the $nth$ bit being 0,
and $p_n(v|1)$ is the probability density function of the received symbol observations $v$,
conditional on the $nth$ bit being 1.

The LLR is an expression of confidence that the transmitted bit is a 1 or 0.
Large positive values suggest a strong likelihood that the transmitted bit was a 1.
Large negative values suggest a strong likelihood that the transmitted bit was a 0.

Note that some literature on the LLR uses a different definition which swaps 0 and 1.
In practical terms these definitions differ only in sign i.e. one definition is the negative of the other.
In this analysis we will use the same convention used by the WSJT-X source code.

### Sum-Product Algorithm

Many soft decision decoders for LDPC codes use variants of an iterative message passing technique
called the sum-product algorithm.
Gallager's original 1960s work proposed a LDPC decoder that is one such variant.

The sum-product algorithm is also known by the name of "belief propagation", 
and this is the name used within the WSJT-X source code,
where the algorithm is implemented in the  `lib/ft8/bpdecode171_91.f90` compilation unit.

The tutorial paper by Kschischang et. al. provides a good history of the sum-product algorithm
and its applicability to a broad range of problems including the decoding of LDPC codes
[[5]](#References).
Key points from this paper are summarised below.

#### Factor Graphs

The sum-product algorithm is most easily described using a concept called the factor graph.

Factor graphs were conceived in an attempt to unify similar concepts that had
independently evolved across multiple disciplines.
They are a generalization of Tanner graphs, which had previously been used to describe the sum-product
algorithm in the information theory literature.

Factor graphs are bipartate graphs that can be used to represent many different types kinds of codes,
of which LDPC codes are an example.

Bipartate graphs are graphs comprised of two sets of nodes, such that graph edges only exist between nodes
in different sets i.e. nodes in the same set can not be directly connected by an edge.

When factor graphs are used to represent LDPC codes, the two sets of nodes in the factor graph are:

1. Bit (variable) nodes which correspond to the bits in the LDPC codeword.
For the FT8 digital mode, there are 174 bit nodes in the factor graph
corresponding to the 174 bits in each LDPC codeword.
2. Check (factor) nodes which correspond to the parity check equations in the LDPC code parity check matrix.
For the FT8 digital mode, there are 83 check nodes in the factor graph corresponding to the 83 parity
check equations.

Where a parity check equation includes a particular bit position, the factor graph has an edge between the bit node that corresponds to the bit postition and the check node corresponding to the
parity check equation.
The LDPC code used by the FT8 digital mode has 522 edges connecting bit nodes with check nodes.
There are 3 edges connected to each symbol node, and either 6 or 7 edges connected to each parity check node.

This LDPC code is called an irregular LDPC code because the number of edges connected to each check
node is not the same for all check nodes.

WSJT-X represents the factor graph for the LDPC code used by the FT8 digital mode using definitions contained
in the `lib/ft8/ldpc_174_91_c_reordered_parity.f90` compilation unit.

#### Algorithm Description

The sum-product algorithm is a message passing algorithm, 
which operates by exchanging messages between nodes along the edges of a factor graph.
When the algorithm is applied to decoding a LDPC code, the factor graph is constructed
using the parity check equations for the LDPC code as described previously.

The algorithm starts by assigning values to each of the bit nodes.
The values assigned to each bit node are the log likelyhood ratios (LLRs) that were calculated
by the detector for each of the corresponding bits in the LDPC codeword.

For factor graphs that contain cycles (as is usually the case for LDPC codes), the algorthm is iterative.
During each iteration, bit nodes send messages to the check nodes to which they are connected on the factor
graph, and the check nodes then send messages back to the bit nodes to which they are connected on the
factor graph.

At each node a calculation is performed on the incoming messages to generate outgoing messages.
A slightly different calculation is performed to generate each outgoing message, depending on the destination
node for that outgoing message.
More specifically, an outgoing message destined to a node is based only on incoming messages
from other nodes i.e. the prior incoming message from the destination node is excluded from the calculation
of the outgoing message to that node.

At each bit node the outgoing message to check node $i$ is calculated by adding the incoming
messages from all check nodes other than check node $i$ to the initial LLR for bit node.

These messages can be considered to be an improved estimate of the LLR for the bit corresponding to the bit
node using corrections from the check nodes other than the check node to which the message is being sent.

In the first iteration of the algorithm there are no incoming messages at the bit nodes, and the outgoing
messages from each bit node are just the LLR that was initially assigned to that bit node i.e. the check nodes
connected to a bit node on the factor graph are all sent the same uncorrected LLR as the message.

At the check nodes the outgoing messages to bit node $j$ are calculated from the
incoming messages from all the other bit nodes using the formula:

$$outmsg_j = -2 \arctan{\left(\prod_{k \neq j} \tan{\left( -inmsg_k/2 \right)} \right)}$$

Note that in some of the information theory literature a slightly different form of the formula is stated
that excludes the minus signs.
This is due to differences in how the LLR is defined in that literature.
The form used here is that used in the WSJT-X source code.

The message sent by the check nodes can be considered to be a correction for the destination bit node.
The correction is based on the corrected LLRs received from other bit nodes in the same parity check equation.

At the start of each iteration the decoder checks if a valid LDPC codeword can be obtained from a hard
detection of the corrected LLR at each bit node.
The corrected LLR used for hard detection includes corrections from all connected check nodes.
A positive corrected LLR is detected as a 1 and a negative corrected LLR is detected as a 0.

The validity of a corrected codeword is checked using the parity check equations for the LDPC code.
The codeword is valid if all the parity check equations are satisfied.
If the codeword is valid the sum-product algorithm would normally terminate at that point having
successfully found a valid LDPC codeword.

However, in the WSJT-X version of the algorithm, an additional test is performed on the codeword using
the cyclic redundancy check (CRC) that was appended to the original message before it was encoded.
If the CRC fails the algorithm continues in the hope that a different valid LDPC codeword will
subsequently be found that does pass the CRC.
If the CRC passes the algorithm terminates with a high level of confidence that an error free message
has been successfully decoded.

To avoid the algorithm iterating indefinitely, the algorithm also terminates after a maximum interation
count is reached.

A further optimisation used by WSJT-X is to terminate the algorithm before the maximum iteration
count if it is determined that the algorithm is not converging on a solution.
This determination is made using an empirical criteria based on the number of unsatisfied parity equations
and how many times this number increased over an iteration.

### Order Statistics Decoder

WSJT-X includes a second soft decision decoder was originally developed in the mid-1990s,
and is known as the ordered statistics decoder [[6](#References)].

This decoder is only used by WSJT-X when "deep" decoding is enabled in an attempt to decode signals that
have not been decoded by the sum-product algorithm.

The order statistics decoder will not be explored further in this notebook.

## CRC Codes

The FT8 digital mode uses a CRC (Cyclic Redundancy Check) code in addition to the LDPC code.
While LDPC codes can both detect and correct errors, CRC codes are generally only used to detect errors.
At very low signal to noise ratios, there is a significant non-zero probability that some errors will not be detected and corrected by the LDPC code used by the FT8 digital mode.
The use of an additional CRC outer code enable detection of many such errors.

Messages found by the CRC code to contain errors are discarded, as the CRC code is not capable of error correction.
The FT8 digital mode will usually retransmit messages until they are successfully acknowledged, so a discarded message will eventually be delivered if the signal to noise ratio is sufficient to allow subsequent delivery of an error free message.

CRC codes are derived from work on cylic error correcting codes in the late 1950s,
and their applicability to error detection was popularized by Peterson and Brown in 1961 [[7](#References)].
These codes have been widely used ever since, particularly in computer communicaton networks where the codes are part of many pervasive computer communication standards e.g. Ethernet.
They are popular primarily because they can easily be implemented in hardware.

Like LDPC codes, CRC codes take a block of message bits and append redundant parity bits to the message bits to create a codeword that is subsequently transmitted using an appropriate modulation method.
The parity bits in CRC codes are the remainder of a polynomial division operation modulo 2,
where the dividend is a polynomial derived from the block of message bits,
and the divisor is a polynomial that is specific to a particular CRC code.

The FT8 data mode uses a $14$ bit CRC code i.e. $14$ redundant parity bits are appended to each block of $77$ message bits to create a $91$ bit codeword.
The $14$ bit CRC is obtained from the remainder of dividing modulo 2 a polynomial derived from the block of $77$
message bits, by the polynomial:

$$x^{14}+x^{13}+x^{10}+x^9+x^8+x^6+x^4+x^2+x^1+1$$

CRC polynomials are often described in hexadecimal after dropping either the first or last term as those terms are always one.
The boost C++ library (used by the WSJT-X program to generate and validate the CRC code)
assumes the first term has been dropped and expresses the FT8 CRC polynomial in hexadecimal as `0x2757`.
Literature that drops the last +1 term expresses the same polynomial as `0x33ab`.

The input to the CRC code is the $77$ bit packed message padded with $19$ trailing zeros to create an array
of $12$ bytes in big endian order.

Once the CRC is calculated, it is appended to the $77$ bit packed message to create $91$ bits for subsequent LDPC encoding.

Upon receipt, the $77$ bit packed message and $14$ bit CRC are extracted from the first $91$ bits of a successfully decoded LDPC codeword.
The $77$ bit message is padded with $19$ trailing zeros and the CRC calculated.
If it matches the received CRC the message is assumed to be error free.

## Gray Codes

Gray codes are named after Frank Gray who first described them in a 1947 patent application [[8](#References)].
Gray codes are generally used to minimise bit errors when converting from the analog to digital domain.
The intent of Gray codes is to choose a mapping between analog and digital representations so that small
errors in the analog domain only cause a minimal number of bit errors in the digital domain.

The FT8 digital mode uses M-GFSK modulation where $M$ is $8$.
This form of modulation conveys symbols rather than individual bits - there are $M$ possible symbols
transmitted, where each symbol represents $\log_2{M}$ bits.
In FT8 there are $8$ possible symbols, so each symbol conveys $\log_2{8} = 3$ bits.

In M-GFSK modulation, each of the possible symbols are transmitted using one of $M$ possible tones (frequencies).
The Gray code used by FT8 defines the mapping between symbol values and transmitted tones
i.e. it determines what tone is transmitted for each of the possible symbol values.
The Gray code is therefore a simple permutation of the $M$ possible symbol values.
Unlike the LDPC and CRC codes, Gray codes do not add any redundancy.
 
The Gray code used by FT8 is defined in the `lib/ft8/genft8.f90` compilation unit as the permutation
$[0, 1, 3, 2, 5, 6, 4, 7]$.
At the receiver this becomes the following mapping from tones to bits:

$$[0, 1, 2, 3, 4, 5, 6, 7] \Rightarrow [000, 001, 011, 010, 110, 100, 101, 111]$$

The key aspect of this mapping is that adjacent tones differ in the eventual binary representation
by only one bit.
This means that if a tone is detected incorrectly as an adjacent tone, it only causes a single bit error.

## References

1. Marcel J. E. Golay, "Notes on Digital Coding," Proceedings of the IRE, vol. 37, p. 657, Jun. 1949.
1. R. W. Hamming, "Error Detecting and Error Correcting Codes," The Bell System Technical Journal, vol. 29, pp. 147-160, Apr. 1950.
1. R. G. Gallager, "Low-Density Parity-Check Codes," IRE Transactions on Information Theory, vol. 8, pp. 21-28, Jan. 1962.
1. David J. C. MacKay and Radford M. Neal, "Near Shannon Limit Performance of Low Density Parity Check Codes," Electronics Letters, vol. 33, pp. 457-458, Mar. 1997.
1. Frank R. Kschischang, Brendan J. Frey and Hans-Andrea Loeliger, "Factor Graphs and the Sum-Product Algorithm", IEEE Transactions on Information Theory, vol. 47, pp. 498-519, Feb. 2001.
1. Marc P. C. Fossorier and Shu Lin, "Soft-Decision Decoding of Linear Block Codes Based on Ordered Statistics", IEEE Transactions on Information Theory, vol. 41, pp. 1379-1396, Sep. 1995.
1. W. W. Peterson and D. T. Brown, "Cyclic Codes for Error Detection,", Proceedings of the IRE, vol. 49, pp. 228-235, Jan. 1961.
1. Frank Gray, "Pulse Code Communication," U.S. Patent 2,632,058, filed 13 Nov. 1947, issued 17 Mar. 1953.

## License
Copyright (C) 2019 James Kelly, VK3JPK.

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.